<a href="https://colab.research.google.com/github/hoangduc143/LinkPrediction_KLTN_2022/blob/main/LinkPredictionFeatherDeezerSocial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Khai báo thư viện
import networkx as nx
import random
from sklearn.metrics import recall_score , precision_score, f1_score

In [2]:
# Liên kết driver
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive/MyDrive/Colab Notebooks

/content/gdrive/MyDrive/Colab Notebooks


In [4]:
# Đọc file dữ liệu lastfm_asia_edges.csv
# Tạo danh sách các đỉnh
file = open('/content/gdrive/MyDrive/Colab Notebooks/LinkPrediction/data/feather-deezer-social/deezer_europe/deezer_europe_edges.csv','r')
edges = []
for line in file.readlines()[1:]:
    tokens = line.strip().split(',')
    edge = (int(tokens[0]),int(tokens[1]))
    edges.append(edge)

In [5]:
# Tạo đồ thị cho dữ liệu
graph = nx.from_edgelist(edgelist=edges)

In [7]:
# # đồ thị các đỉnh
# nx.draw_networkx_nodes(graph,pos=nx.spring_layout(graph),node_shape='.',linewidths=0.1)
# # đồ thị (graph)
# nx.draw(graph)
# # đồ thị các cạnh
# nx.draw_networkx_edges(graph)

In [8]:
# In ra số cạnh và số đỉnh
print("Số đỉnh:",len(graph.nodes))
print("Số cạnh:",len(graph.edges))

Số đỉnh: 28281
Số cạnh: 92752


In [9]:
# Chọn cứng trình tạo số ngẫu nhiên
random.seed(10)

In [10]:
# Tỉ lệ số cạnh bị xóa đi trong đồ thị
def remove_edges(graph,name_node, ratio_remove=0.5):
  remove_nodes = set(random.sample([n for n in graph[name_node]],round(ratio_remove*len(graph[name_node]))))
  for n in remove_nodes:
    graph.remove_edge(name_node,n)
  candidates = [(name_node, n) for n in graph if n not in graph[name_node]]
  return remove_nodes , graph , candidates



In [11]:
remove_nodes, graph , candidates = remove_edges(graph,2)

In [12]:
def jaccard(graph , candidates,name_node, top=20):
  scores = [e for e in nx.jaccard_coefficient(graph,candidates)]
  scores.sort(key=lambda e:e[2],reverse=True)
  predictions = set([e[1] for e in scores[:top]])
  return predictions

def common_neighbors(graph , candidates,name_node, top=20):
  scores = [e for e in nx.common_neighbor_centrality(graph,(2, 1126))]
  scores.sort(key=lambda e:e[2],reverse=True)
  predictions = set([e[1] for e in scores[:top]])
  return predictions

def adamic_adar(graph , candidates,name_node, top=20):
  scores = [e for e in nx.adamic_adar_index(graph,candidates)]
  scores.sort(key=lambda e:e[2],reverse=True)
  predictions = set([e[1] for e in scores[:top]])
  return predictions

def pagerank(graph , candidates,name_node, top=20):
  scores = [e for e in nx.pagerank(graph,personalization={ name_node: 1}, alpha=0.9).items() ]
  scores.sort(key=lambda e:e[1],reverse=True)
  predictions = set([e[0] for e in scores[:top]])
  return predictions

def score(graph,name_node):
  remove_nodes, graph_after , candidates = remove_edges(graph,name_node)
  print("Tên node đang xet: ",name_node)
  print("Các node đã xóa: ")
  print(remove_nodes)
  print("Tập dự đoán sử dụng jaccard score:")
  print(jaccard(graph_after , candidates, name_node))
  print("Các dự đoán đúng của jaccard score:")
  print(jaccard(graph_after , candidates, name_node).intersection(remove_nodes))
  
  print("Tập dự đoán sử dụng adamic_adar score:")
  print(adamic_adar(graph_after , candidates, name_node))
  print("Các dự đoán đúng của adamic_adar score:")
  print(adamic_adar(graph_after , candidates, name_node).intersection(remove_nodes))
  
  print("Tập dự đoán sử dụng pagerank score:")
  print(pagerank(graph_after , candidates,name_node))
  print("Các dự đoán đúng của pagerank score:")
  print(pagerank(graph_after , candidates,name_node).intersection(remove_nodes))

  # print(recall_score(remove_nodes, jaccard(graph_after , candidates, name_node)))
  # print(precision_score(remove_nodes, jaccard(graph_after , candidates, name_node)))
  # print(f1_score(remove_nodes, jaccard(graph_after , candidates, name_node)))

In [13]:
name_node = 22
score(graph,name_node)

Tên node đang xet:  22
Các node đã xóa: 
{7659, 2261}
Tập dự đoán sử dụng jaccard score:
{0, 6288, 22, 15894, 6552, 16414, 11812, 25525, 3001, 27579, 14270, 14145, 16976, 25564, 22120, 4075, 10990, 14581, 19832, 12029}
Các dự đoán đúng của jaccard score:
set()
Tập dự đoán sử dụng adamic_adar score:
{0, 6288, 22, 15894, 6552, 16414, 11812, 25525, 3001, 27579, 14270, 14145, 16976, 25564, 22120, 4075, 10990, 14581, 19832, 12029}
Các dự đoán đúng của adamic_adar score:
set()
Tập dự đoán sử dụng pagerank score:
{7172, 4877, 6288, 22, 15894, 6552, 16414, 11812, 27945, 16948, 25525, 9528, 27579, 2369, 21849, 25689, 22120, 4075, 10990, 19832}
Các dự đoán đúng của pagerank score:
set()


In [14]:
# code cho common_neighbor_centrality 
G = nx.complete_graph(5)
preds = nx.common_neighbor_centrality(G, [(0, 1), (2, 3)])
for u, v, p in preds:
    print(f"({u}, {v}) -> {p}")

(0, 1) -> 3.4000000000000004
(2, 3) -> 3.4000000000000004


In [ ]:
# Code cho simrank
G = nx.cycle_graph(1000)
nx.simrank_similarity(G)

nx.simrank_similarity(graph)